### Ce Notebook permet de convertir les fichiers d'entrainement Spacy2 en fichiers d'entrainement binaires Spacy3


- Ce notebook permet aussi suite à l'entrainement qui se fait dans un terminal de charger le modèle enregistré et d'évaluer sur les titres des recettes

In [1]:
import spacy
from spacy.tokens import DocBin
import json
from tqdm import tqdm
import random

In [2]:
#!pip install srsly

In [3]:
# Méthode pour lire un fichier json
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

### Chargement des data train/test/eval pour conversion en .spacy

In [4]:

ner_train = load_data("data/ner_training_data.json")

In [5]:
print('Dimension du data set : ',len(ner_train),'  pour le train \n\n')

Dimension du data set :  172727   pour le train 




In [6]:
 print (ner_train[1])

['mélang épic tandoor épic indien roug lait yaourt grec huil jus citron', {'entities': [[0, 6, 'ACTION'], [7, 11, 'INGREDIENT'], [12, 19, 'USTENSILE'], [20, 24, 'INGREDIENT'], [25, 31, 'INGREDIENT'], [37, 41, 'INGREDIENT'], [42, 48, 'INGREDIENT'], [49, 53, 'INGREDIENT'], [54, 58, 'INGREDIENT'], [59, 62, 'INGREDIENT'], [63, 69, 'INGREDIENT']]}]


In [20]:
ner_test = load_data("data/ner_test_data.json")#ner_valid
print('Dimension du data set : ',len(ner_test),'  pour le test \n\n')

Dimension du data set :  43095   pour le test 




In [8]:
nerTitr_valid = load_data("data/nerTitre_test_data.json") 
print('Dimension du data set : ',len(nerTitr_valid),'  pour l\'éval sur les titres \n\n')

Dimension du data set :  22074   pour l'éval sur les titres 




In [9]:
# Méthode pour conversion du format Spacy2 au format Spacy 3

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA): 
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [10]:
ner_train = create_training(ner_train)
ner_train.to_disk("./data/ner_train.spacy")

100%|██████████| 172727/172727 [03:45<00:00, 767.14it/s]


In [11]:
 
ner_valid = create_training(ner_valid)
ner_valid.to_disk("./data/ner_valid.spacy")

100%|██████████| 43095/43095 [00:50<00:00, 859.12it/s]


In [12]:

nerTitr_eval = create_training(nerTitr_valid)
nerTitr_eval.to_disk("./data/nerTitr_eval.spacy")

100%|██████████| 22074/22074 [00:20<00:00, 1068.46it/s]


 ### On charge le meilleur modèle sauvegardé

In [13]:
from spacy.ml.models.tok2vec import MultiHashEmbed

In [14]:
#load the best model
nlp1 = spacy.load(R".\output\model-best") 

In [15]:
doc=nlp1('lasagn végétarien facil  terrin foi porc  lapin vin blanc cocott  escalop lucullus cocott  tart oignon carr coq vin facil ')

In [16]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [17]:
# Prédiction
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

### On évalue: on visualise les éléments taggués comme ingrédients 

In [19]:
for txt in nerTitr_valid:   
    selected_text = predict_entities(txt[0],nlp1)
    print(selected_text)


agneau
cook
chinois
porc
saindoux
quich
asiat
magret
brioch
tart
noix
frais
tart
pomm
mouss
flan
tart
poulet
avocat
quenel
gratin
sauc
fruit
moul
compot
rôt
spaëtzl
tart
épinard
lapin
pudding
salad
veau
cak
tartar
pain
gratin
truit
flan
souffl
crevet
mer
glac
charlott
frambois
tart
cak
boeuf
pain
pizz
fruit
rôt
verrin
gâteau
crem
nem
tart
coup
spaghet
quich
canet
glac
escalop
soup
filet
verrin
crem
foi
frais
tartin
tart
papay
bavarois
magret
gâteau
tartin
choux
madelein
filet
fan
gâteau
yaourt
potiron
volaill
yaourt
pât
cuiss
jambon
dind
gâteau
verrin
papillot
cak
tart
pen
jambon
risotto
cabillaud
pain
pomm
salad
salad
cannel
pissenlit
gratin
tart
poulet
entrecôt
four
tart
tart
yaourt
flan
tart
papillot
saumon
saut
salad
poulet
hach
gâteau
quich
terrin
biscuit
poisson
blanquet
crem
asperg
pomm
sauc
gâteau
quich
tagliatel
charlott
chorizo
mang
verrin
gâteau
terrin
moul
croqu
quich
haricot
pât
terrin
roul
galet
côt
gâteau
muffin
spaghet
bec
poireau
frais
gâteau
gratin
salad
plat
salad
f

In [21]:
for txt in ner_test:   
    selected_text = predict_entities(txt[0],nlp1)
    print(selected_text)

badigeon
gruyer
mélang
23cm
2em
2
badigeon
enfourn
salad
1
lav
coup
2
3
réserv
reven
sal
reven
color
viand
serv
plat
1
chapelur
cuiller
réserv
2
lav
coup
coup
huil
poivron
étal
légum
poivron
coup
pel
saladi
assaison
coriandr
pât
serv
pât
typepad
ros
coup
beurr
courget
préchauff
courget
plat
enfourn
four
sal
serv
blend
cannel
farin
50 cl
moul
coup
margarin
concentr
graiss
plat
saupoudr
couvr
serv
cabillaud
vin
concombr
coup
mergu
coup
concombr
laiss
jus
laiss
ratatouill
1
poulet
film
2
oignon
eau
mélang
couvr
eau
coup
plong
coup
eau
poêl
assaison
jus
serv
riz
prépar
prépar
pain
pât
assaison
pain
filet
enfourn
filet
croût
ros
jus
choucrout
cocott
zest
viand
press
1
laiss
casserol
eau
20
choux
choux
lait
sal
choux
prépar
four
légum
légum
prépar
port
incorpor
yaourt
lait
incorpor
pot
yaourt
lait
rectifi
chocolat
bouill
infus
vanill
ébullit
mélang
prépar
mélang
crem
saupoudr
serv
coup
oignon
oignon
pât
oignon
poivr
dépos
four
serv
préchauff
beurr
réserv
saladi
battr
incorpor
pât
enfourn
dém

KeyboardInterrupt: 

In [24]:
var = ner_test [3]
var
doc=nlp1('démoul gâteau pein sort four remettr moul')

In [25]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [27]:
var = ner_test [4]
var
doc=nlp1('démoul gâteau pein sort four remettr moul')
spacy.displacy.render(doc, style="ent", jupyter=True)

In [32]:
 ner_test [15]

['coup viand porc dé fair reven mijot cocott feu huil arachid couvert',
 {'entities': [[0, 4, 'ACTION'],
   [5, 10, 'INGREDIENT'],
   [11, 15, 'INGREDIENT'],
   [24, 29, 'ACTION'],
   [30, 35, 'ACTION'],
   [36, 42, 'USTENSILE'],
   [47, 51, 'INGREDIENT'],
   [52, 59, 'INGREDIENT']]}]

In [33]:
doc=nlp1('coup viand porc dé fair reven mijot cocott feu huil arachid couvert')
spacy.displacy.render(doc, style="ent", jupyter=True)

In [38]:
 ner_test [50]

['vers filet huil noiset beurr',
 {'entities': [[5, 10, 'INGREDIENT'],
   [11, 15, 'INGREDIENT'],
   [16, 22, 'INGREDIENT'],
   [23, 28, 'INGREDIENT']]}]

In [35]:
doc=nlp1('mettr calvados poêl temp bol mélang maïzen eau froid ajout crem fraîch')
spacy.displacy.render(doc, style="ent", jupyter=True)